<a href="https://www.kaggle.com/code/nigamshitij/batch-processing-of-genres?scriptVersionId=193846027" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-genres/split_dataframe_2.csv
/kaggle/input/book-genres/split_dataframe_32.csv
/kaggle/input/book-genres/split_dataframe_30.csv
/kaggle/input/book-genres/split_dataframe_47.csv
/kaggle/input/book-genres/split_dataframe_37.csv
/kaggle/input/book-genres/split_dataframe_80.csv
/kaggle/input/book-genres/split_dataframe_27.csv
/kaggle/input/book-genres/split_dataframe_105.csv
/kaggle/input/book-genres/split_dataframe_122.csv
/kaggle/input/book-genres/split_dataframe_112.csv
/kaggle/input/book-genres/split_dataframe_63.csv
/kaggle/input/book-genres/split_dataframe_54.csv
/kaggle/input/book-genres/split_dataframe_121.csv
/kaggle/input/book-genres/split_dataframe_115.csv
/kaggle/input/book-genres/split_dataframe_83.csv
/kaggle/input/book-genres/split_dataframe_58.csv
/kaggle/input/book-genres/split_dataframe_94.csv
/kaggle/input/book-genres/split_dataframe_96.csv
/kaggle/input/book-genres/split_dataframe_35.csv
/kaggle/input/book-genres/split_dataframe_64.csv
/kaggle/input/bo

In [3]:
pip install beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import RequestException
import time
import os

In [13]:
# genre setup

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}

items_per_page = 100
# total_pages = (total_items + items_per_page - 1)
total_pages = 14
# max_pages_to_scrape = 14

In [14]:
csv_path = '/kaggle/input/book-genres/'

def scrape_goodreads_genre(genre, base_url, max_pages_to_scrape=25, max_retries=3, retry_delay=5):
    books_data = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }
    
    base_url = "https://www.goodreads.com/shelf/show/"+genre
    
    for page in range(1, max_pages_to_scrape + 1):
        url = f"{base_url}?page={page}"
        retries = 0
        while retries < max_retries:
            try:
                response = requests.get(url, headers=headers, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")
                book_elements = soup.find_all("div", "elementList")
                
                if not book_elements:
                    print(f"No more books found for {genre} after page {page-1}")
                    return pd.DataFrame(books_data)
                
                for book_element in book_elements:
                    try:
                        book_title = book_element.find("a", "bookTitle").text.strip()
                        book_url = "https://www.goodreads.com" + book_element.find("a", "bookTitle").get("href")
                        author = book_element.find("a", "authorName").text.strip()
                        rating_text = book_element.find("span", "greyText smallText").text.split()
                        avg_rating = rating_text[2]
                        ratings = rating_text[4]
                        published_year = rating_text[-1] if len(rating_text) == 9 else ""
                
                        books_data.append({
                            "Title": book_title,
                            "URL": book_url,
                            "Authors": author,
                            "Avg Ratings": avg_rating,
                            "Rating": ratings,
                            "Published_year": published_year,
                            "Genre": genre
                        })
                    except AttributeError:
                        pass
                
                break  # If successful, break out of the retry loop
            
            except RequestException as e:
                retries += 1
                print(f"Error on page {page} of {genre}: {str(e)}. Retry {retries}/{max_retries}")
                if retries == max_retries:
                    print(f"Failed to retrieve page {page} for {genre} after {max_retries} attempts. Moving to next page.")
                    break
                time.sleep(retry_delay)
        
        time.sleep(1)  # Delay between successful page requests
    
    return pd.DataFrame(books_data)

def process_csv_files(num_files=125):
    all_genres_data = []
    
    for i in range(1, num_files + 1):
        file_name = f"split_dataframe_{i}.csv"
        print(f"Processing file: {file_name}")
        
        file_name = csv_path + file_name
        
        # Read the genres from the CSV file
        genres_df = pd.read_csv(file_name)
        
        # Loop through genres and scrape data
        for index, row in genres_df.iterrows():
            genre = row['Genre']
            url = row['URL']
            
            print(f"Scraping data for {genre} genre...")
            df = scrape_goodreads_genre(genre, url)
            all_genres_data.append(df)
            print(f"Finished scraping {genre}. DataFrame shape: {df.shape}")
            print("\n" + "="*50 + "\n")
        
        # Combine all genre DataFrames into a single DataFrame
        combined_df = pd.concat(all_genres_data, ignore_index=True)
        
        # Remove duplicates based on Title and Authors
        combined_df.drop_duplicates(subset=['Title', 'Authors'], keep='first', inplace=True)
        
        # Save the combined DataFrame to a CSV file
        output_file = f"goodreads_genres_no_duplicates_{i}.csv"
        combined_df.to_csv(output_file, index=False)
        print(f"Data from file {file_name} has been processed and saved to '{output_file}'.")
        print(f"Total number of unique books scraped so far: {len(combined_df)}")
        print("\n" + "="*50 + "\n")

# Run the process
process_csv_files()

# After processing all files, combine them into a single CSV
all_data = []
for i in range(1, 126):
    file_name = f"goodreads_genres_no_duplicates_{i}.csv"
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
        all_data.append(df)

final_df = pd.concat(all_data, ignore_index=True)
final_df.drop_duplicates(subset=['Title', 'Authors'], keep='first', inplace=True)
final_df.to_csv("goodreads_all_genres_final.csv", index=False)

print("All CSV files have been processed and combined.")
print(f"Final data has been saved to 'goodreads_all_genres_final.csv'.")
print(f"Total number of unique books in the final dataset: {len(final_df)}")

Processing file: split_dataframe_1.csv
Scraping data for 10th-century genre...
Error on page 13 of 10th-century: HTTPSConnectionPool(host='www.goodreads.com', port=443): Read timed out. (read timeout=10). Retry 1/3
Finished scraping 10th-century. DataFrame shape: (1250, 7)


Scraping data for 11th-century genre...
Finished scraping 11th-century. DataFrame shape: (1250, 7)


Scraping data for 12th-century genre...
Finished scraping 12th-century. DataFrame shape: (1250, 7)


Scraping data for 13th-century genre...
Finished scraping 13th-century. DataFrame shape: (1250, 7)


Scraping data for 14th-century genre...
Finished scraping 14th-century. DataFrame shape: (1250, 7)


Scraping data for 15th-century genre...


KeyboardInterrupt: 